In [22]:
# Step 1: Import Required Libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report


In [23]:
# Step 2: Load IMDb Dataset (top 10000 frequent words)
vocab_size = 10000
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size)

# Step 3: Pad sequences to same length
maxlen = 300
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

In [24]:
# Step 4: Define the RNN Model (stacked with dropout)
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=64, input_length=maxlen))  # Embedding Layer
model.add(SimpleRNN(64, return_sequences=True))                                 # First RNN Layer
model.add(Dropout(0.3))
model.add(SimpleRNN(32))                                                        # Second RNN Layer
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))                                       # Output Layer


C:\Users\91862\anaconda3\lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [25]:
# Step 5: Compile the model with Adam optimizer
from tensorflow.keras.optimizers import Adam
model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0003), metrics=['accuracy'])


In [26]:
# Step 6: Train the model with EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

print("Training RNN...")
model.fit(x_train, y_train, epochs=10, batch_size=64, validation_split=0.2, callbacks=[early_stop])

Training RNN...
Epoch 1/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 101s 305ms/step - accuracy: 0.6324 - loss: 0.6265 - val_accuracy: 0.8174 - val_loss: 0.4234
Epoch 2/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 85s 271ms/step - accuracy: 0.8714 - loss: 0.3348 - val_accuracy: 0.8518 - val_loss: 0.3436
Epoch 3/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 97s 309ms/step - accuracy: 0.9144 - loss: 0.2369 - val_accuracy: 0.8668 - val_loss: 0.3200
Epoch 4/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 136s 291ms/step - accuracy: 0.9421 - loss: 0.1713 - val_accuracy: 0.8710 - val_loss: 0.3580
Epoch 5/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 82s 263ms/step - accuracy: 0.9538 - loss: 0.1421 - val_accuracy: 0.8658 - val_loss: 0.3842


In [27]:
# Step 7: Evaluate on test data
print("\nEvaluating on test data...")
loss, accuracy = model.evaluate(x_test, y_test)

# Step 8: Predict and show classification report
y_pred = (model.predict(x_test) > 0.5).astype("int32")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("Test Accuracy:", accuracy)


Evaluating on test data...
782/782 ━━━━━━━━━━━━━━━━━━━━ 33s 42ms/step - accuracy: 0.8609 - loss: 0.3394
782/782 ━━━━━━━━━━━━━━━━━━━━ 26s 33ms/step

Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.87      0.86     12500
           1       0.87      0.85      0.86     12500

    accuracy                           0.86     25000
   macro avg       0.86      0.86      0.86     25000
weighted avg       0.86      0.86      0.86     25000

Test Accuracy: 0.8621600270271301


In [28]:
# Step 9: Test on a few examples
print("\nSample Predictions:")
for i in range(5):
    print("True label:", y_test[i], "Predicted:", y_pred[i][0])


Sample Predictions:
True label: 0 Predicted: 0
True label: 1 Predicted: 1
True label: 1 Predicted: 1
True label: 0 Predicted: 0
True label: 1 Predicted: 1
